# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import pymongo

%matplotlib inline

In [3]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [4]:
class MongoHandler():
    pass

In [2]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
pd.read_sql_query("""SELECT * FROM Teams WHERE Season = 2011""", conn)

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2011,Bayern Munich,24,25,11,335600000,13980000,75000
1,2011,Dortmund,28,24,12,158200000,5650000,81359
2,2011,Leverkusen,29,23,9,151100000,5210000,30210
3,2011,Schalke 04,37,24,20,136730000,3700000,62271
4,2011,Werder Bremen,38,23,17,125350000,3300000,42100
5,2011,Wolfsburg,44,25,28,122000000,2770000,30000
6,2011,Hamburg,36,23,23,103200000,2870000,57376
7,2011,Stuttgart,32,24,18,101880000,3180000,60449
8,2011,Hoffenheim,34,23,20,93600000,2750000,30164
9,2011,FC Koln,31,24,17,69290000,2240000,49968


In [4]:
pd.read_sql_query("""SELECT * FROM Matches LIMIT 10""", conn)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [5]:
pd.read_sql_query("""SELECT * FROM Unique_Teams LIMIT 10""", conn)

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5
5,M'gladbach,6
6,Wolfsburg,7
7,FC Koln,8
8,Hoffenheim,9
9,Hertha,10


In [6]:
pd.read_sql_query("""SELECT * FROM Teams_In_Matches WHERE Match_ID = 44874""", conn)

,Match_ID,Unique_Team_ID
0,44874,119
1,44874,120


In [7]:
FV = pd.read_sql_query("""SELECT * FROM FlatView""", conn)
FV.columns
FV_clean = FV[['Match_ID', 'Div', 'Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTR']]
FV_clean

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,A
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,D
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,H
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,A
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,H
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,H


In [43]:
# FVA = pd.read_sql_query("""SELECT * FROM FlatView_Advanced WHERE Season = '2011' ORDER BY Match_ID""", conn)
# FVA.columns
# FVA_clean = FVA[['Match_ID', 'Unique_Team_ID', 'Unique_Team', 'Div', 'Season', 'Date',
#        'HomeTeam', 'AwayTeam', 'FTR']]
# FVA_clean

In [8]:
FVAChrono = pd.read_sql_query("""SELECT * FROM FlatView_Chrono_TeamOrder_Reduced WHERE Season = '2011' ORDER BY Match_ID""", conn)
FVAChrono 

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
1,1092,22,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Nurnberg
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
3,1093,17,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Stuttgart
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
5,1094,7,Wolfsburg,7,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Wolfsburg
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich
7,1095,11,Mainz,11,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich
9,1096,16,Freiburg,16,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Freiburg


In [76]:
# FVAChrono.drop(['TeamName'], axis=1, inplace=True)

In [9]:
FVAChronoUM = FVAChrono[::2]
FVAChronoUM.head(20)

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich
10,1097,1,Bayern Munich,1,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,Bayern Munich
12,1098,1,Bayern Munich,1,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,Bayern Munich
14,1099,1,Bayern Munich,1,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,Bayern Munich
16,1100,1,Bayern Munich,1,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,Bayern Munich
18,1101,1,Bayern Munich,1,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,Bayern Munich


In [10]:
TotalGoals = FVAChronoUM.groupby("TeamName")['FTHG', 'FTAG'].sum(axis=1)
TotalGoals['Total Goals'] = TotalGoals['FTHG'] + TotalGoals['FTAG']
TotalGoals.head(20)

,FTHG,FTAG,Total Goals
TeamName,,,
Aachen,12,14,26
Arsenal,71,52,123
Aston Villa,44,40,84
Augsburg,12,11,23
Bayern Munich,65,34,99
Blackburn,36,29,65
Bochum,30,19,49
Bolton,24,23,47
Braunschweig,35,25,60


In [16]:
FVAChronoUM

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich
10,1097,1,Bayern Munich,1,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,Bayern Munich
12,1098,1,Bayern Munich,1,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,Bayern Munich
14,1099,1,Bayern Munich,1,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,Bayern Munich
16,1100,1,Bayern Munich,1,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,Bayern Munich
18,1101,1,Bayern Munich,1,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,Bayern Munich


In [82]:
FVAChronoUM['Winner'] = 1

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
FVAChronoUM['Winner'] = FVAChronoUM.where((FVAChronoUM['FTR'] == 'A') | (FVAChronoUM['FTR'] == 'D'), 
                                          FVAChronoUM['HomeTeam'], axis = 0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [84]:
FVAChronoUM['Winner'] = FVAChronoUM['Winner'].where((FVAChronoUM['FTR'] == 'D') | (FVAChronoUM['FTR'] == 'H'), 
                                                    FVAChronoUM['AwayTeam'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [85]:
FVAChronoUM['Winner'] = FVAChronoUM['Winner'].where((FVAChronoUM['FTR'] == 'A') | (FVAChronoUM['FTR'] == 'H'), 
                                                    'Draw')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [93]:
FVAChronoUM.head(20)
# Winsdf = pd.DataFrame(FVAChronoUM['Winner'].value_counts())
# Winsdf

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team,Winner,Loser
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,1
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,1
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,1
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich,Mainz,1
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich,Draw,1
10,1097,1,Bayern Munich,1,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,Bayern Munich,M'gladbach,1
12,1098,1,Bayern Munich,1,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,Bayern Munich,Draw,1
14,1099,1,Bayern Munich,1,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,1
16,1100,1,Bayern Munich,1,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,Bayern Munich,Bayern Munich,1
18,1101,1,Bayern Munich,1,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,Bayern Munich,Hannover,1


In [102]:
FVAChronoUM['Loser'] = 1

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
FVAChronoUM['Loser'] = FVAChronoUM.where((FVAChronoUM['FTR'] == 'A') | (FVAChronoUM['FTR'] == 'D'), 
                                          FVAChronoUM['AwayTeam'], axis = 0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [106]:
FVAChronoUM['Loser'] = FVAChronoUM['Loser'].where((FVAChronoUM['FTR'] == 'D') | (FVAChronoUM['FTR'] == 'H'), 
                                                    FVAChronoUM['HomeTeam'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
FVAChronoUM['Loser'] = FVAChronoUM['Loser'].where((FVAChronoUM['FTR'] == 'A') | (FVAChronoUM['FTR'] == 'H'), 
                                                    'Draw')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [107]:
FVAChronoUM

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team,Winner,Loser
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,Nurnberg
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,Stuttgart
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,Wolfsburg
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich,Mainz,Mainz
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich,Draw,Draw
10,1097,1,Bayern Munich,1,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,Bayern Munich,M'gladbach,M'gladbach
12,1098,1,Bayern Munich,1,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,Bayern Munich,Draw,Draw
14,1099,1,Bayern Munich,1,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,Werder Bremen
16,1100,1,Bayern Munich,1,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,Bayern Munich,Bayern Munich,Schalke 04
18,1101,1,Bayern Munich,1,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,Bayern Munich,Hannover,Hannover


In [122]:
# FVAChronoUM['Winner'] = FVAChronoUM['Winner'].where(FVAChronoUM['FTR'] == 'A', FVAChronoUM['HomeTeam'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [131]:
# FVAChronoUM['Winner'] = FVAChronoUM['Winner'].where(((FVAChronoUM['FTR'] != 'A') & (FVAChronoUM['FTR'] != 'D')), FVAChronoUM['HomeTeam'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
# FVAChronoUM['Winner']

In [108]:
FVAChronoUM

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team,Winner,Loser
0,1092,1,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,Nurnberg
2,1093,1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,Stuttgart
4,1094,1,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,Bayern Munich,Wolfsburg
6,1095,1,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Bayern Munich,Mainz,Mainz
8,1096,1,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Bayern Munich,Draw,Draw
10,1097,1,Bayern Munich,1,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,Bayern Munich,M'gladbach,M'gladbach
12,1098,1,Bayern Munich,1,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,Bayern Munich,Draw,Draw
14,1099,1,Bayern Munich,1,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,Bayern Munich,Bayern Munich,Werder Bremen
16,1100,1,Bayern Munich,1,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,Bayern Munich,Bayern Munich,Schalke 04
18,1101,1,Bayern Munich,1,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,Bayern Munich,Hannover,Hannover


# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!